# Projet IC05 – Analyse des Top50 Spotify
## Collecte et traitement des données des artistes issus des Top50

### Imports

In [2]:
import requests
import csv
import pandas as pd
from credentials import ClientIDCO, ClientSecretCO

def get_acces_token(client_id, client_secret):
    """retourne un json (dict)"""
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials",
        "client_id": f"{client_id}",
        f"client_secret": f"{client_secret}"
    }

    response = requests.post(url, headers=headers, data=data)

    return response.json()

# recupérer un acces token
AccessToken = get_acces_token(ClientIDCO, ClientSecretCO)
# print(AccessToken['access_token'])





### Collecte des données sur les artistes

In [3]:
# get id list
def json_to_csv(json_data, output_filename):
    # Préparer le fichier CSV
    with open(output_filename, mode='a+', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["artists", "followers", "genres", "name", "popularity"])
        writer.writeheader()

        # Traiter chaque artiste dans la liste
        for artist in json_data["artists"]:
            # Extraire l'ID à partir de l'URL Spotify
            full_url = artist.get("external_urls", {}).get("spotify", "")
            artist_id = full_url.split("/")[-1] if full_url else ""

            # Préparer les informations pertinentes pour chaque artiste
            row = {
                "artists": artist_id,
                "followers": artist.get("followers", {}).get("total", 0),
                "genres": ", ".join(artist.get("genres", [])),
                "name": artist.get("name", ""),
                "popularity": artist.get("popularity", 0)
            }
            writer.writerow(row)


In [5]:

# retrieve artists info (name, genres,popularity, followers)
# faire les requetes par pays
def get_artists_info(AccessToken, src_file, dst_file) :
    

    artists_ids_string = ""
    artists_ids_list = ""
    result =[]

    # lecture du fichier CSV et recup les artistes_id
    with open(src_file, 'r', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        first_row = next(reader)
        for row in reader:
            # print(row[2]) # col artistes
            if "," in row[2] :
                # print(row[2].strip('"[]').replace(" ", ""))
                artists_ids_string += row[2].strip('"[]').replace(" ", "")
            else :
                # print(row[2][1:23])
                artists_ids_string += row[2][1:23]

            artists_ids_string +=","
        
               
    # print(artists_ids_string)
    artists_ids_list = artists_ids_string.split(",")
    # print(artists_ids_list)
    artists_ids_list = list(dict.fromkeys(artists_ids_list))

    # artists_ids_list = "61ydMyohF67xBZmWSPA3YQ,5mPbBOU2KMFfStepNZ027p,5Xn0cJrrcslCWCzPXNTy54,4bPYvyUn1mWwUwIW57oST2,6zUWZmyi5MLOEynQ5wCI5f,1Tdy7LZnrufTyuEbgDi1D8,61ydMyohF67xBZmWSPA3YQ,5mPbBOU2KMFfStepNZ027p,5Xn0cJrrcslCWCzPXNTy54,4bPYvyUn1mWwUwIW57oST2,6zUWZmyi5MLOEynQ5wCI5f,1Tdy7LZnrufTyuEbgDi1D8,61ydMyohF67xBZmWSPA3YQ,5mPbBOU2KMFfStepNZ027p,5Xn0cJrrcslCWCzPXNTy54,61ydMyohF67xBZmWSPA3YQ, 5mPbBOU2KMFfStepNZ027p,5Xn0cJrrcslCWCzPXNTy54,4bPYvyUn1mWwUwIW57oST2,6zUWZmyi5MLOEynQ5wCI5f,1Tdy7LZnrufTyuEbgDi1D8,61ydMyohF67xBZmWSPA3YQ,5mPbBOU2KMFfStepNZ027p,5Xn0cJrrcslCWCzPXNTy54,4bPYvyUn1mWwUwIW57oST2,6zUWZmyi5MLOEynQ5wCI5f,1Tdy7LZnrufTyuEbgDi1D8,61ydMyohF67xBZmWSPA3YQ,5mPbBOU2KMFfStepNZ027p,5Xn0cJrrcslCWCzPXNTy54"
    for i in range(0, len(artists_ids_list), 50 ):
        chunk = artists_ids_list[i:i + 50]  # 50 artistes
        # print(','.join(chunk), "\n")
        response = requests.get(
            url = "https://api.spotify.com/v1/artists?ids=" +','.join(chunk),
            headers = {
                "Authorization": 'Bearer ' + AccessToken['access_token']
            }
        )
        # print(response.json())
        # requête réussie
        if response.status_code == 200:
            json_to_csv(response.json(), dst_file)
        # erreur requete
        else:
            print(f"Erreur lors de la requête pour le lot {i // 50 + 1}: {response.status_code}")

    #return artists_ids_list, artists_ids_list
get_artists_info(AccessToken, "data/Africa.csv", "data/artists/artists_africa.csv")
get_artists_info(AccessToken, "data/Asia.csv", "data/artists/artists_asia.csv")
get_artists_info(AccessToken, "data/Central_america.csv", "data/artists/artists_central_america.csv")
get_artists_info(AccessToken, "data/South_America.csv", "data/artists/artists_south_america.csv")
get_artists_info(AccessToken, "data/North_America.csv", "data/artists/artists_north_america.csv")
get_artists_info(AccessToken, "data/Europe.csv", "data/artists/artists_europe.csv")
get_artists_info(AccessToken, "data/Oceania.csv", "data/artists/artists_oceania.csv")

  
